In [1]:
from hashlib import sha256
import math
from web3 import Web3
import json
import time
from datetime import datetime, timedelta
from web3.exceptions import ContractLogicError
import eth_abi
import random
import solcx
import os
import hashlib
import numpy as np
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util import Counter
import secrets

In [2]:
# This mapping is used to convert a command to an operation
opMap = {
    "add": lambda x, y: x + y,
    "subtract": lambda x, y: x - y,
    "divide": lambda x, y: np.divide(x, y),
    "mul": lambda x, y: x * y,
    "matmul": lambda x, y: np.dot(x, y)
}

# This class represents a node in the DAG
class Node:
    def __init__(self, symbol, data, left=None, right=None, elementCount = 1):
        self.symbol = symbol
        self.data = data
        self.dataList = list(map(lambda x: int(x), data.reshape(-1, )))
        self.size = len(self.dataList) * elementCount
        self.left = left
        self.right = right
        self.elementCount = elementCount # This variable is used to specify the number of operations one must perform to calculate an element in this node

# Convert our inputs into nodes for the DAG
def processInputs(input_data, dag):
    for k, v in input_data.items():
        dag[k] = Node(k, v)

# Solve the DAG. This involves creating nodes and connecting them, and then solving them.
def solveDAG(command, dag, outputs):
    if command.startswith("output"): 
        res, sym = command.split(" ")
        outputs.add(sym)
    else:
        res, op, x, y = command.split(" ")
        dag[res] = Node(res, opMap[op](dag[x].data, dag[y].data), dag[x], dag[y], dag[x].data.shape[1] * 2 - 1 if op == "matmul" else 1)

# operationIdx is the number of operations we've done into the round so far
# roundNum is the number of the round we're currently on
# layerSize is the number of operations we can perform in any given round at most
# rounds is the dictionary that contains the data for each round
# outputs is the set of symbols that are outputs
# root is the root node of the DAG that we are currently solving
def calculateRoundParams(root, roundNum, operationsIdx, rounds, layerSize, outputs):
    if root == None: return [roundNum, operationsIdx]
    if root.left == None and root.right == None: return [roundNum, operationsIdx]

    # we first process the left and right nodes
    roundNum, operationsIdx = calculateRoundParams(root.left, roundNum, operationsIdx, rounds, layerSize, outputs)
    roundNum, operationsIdx = calculateRoundParams(root.right, roundNum, operationsIdx, rounds, layerSize, outputs)

    rootIdx = 0
    if not roundNum in rounds: rounds[roundNum] = {}

    # This loop is used to create the data for each round. We keep looping whilst we are processing this element
    while rootIdx < len(root.dataList):
        newRootIdx = min(math.ceil((layerSize - operationsIdx) / root.elementCount) + rootIdx, len(root.dataList)) # either this is more than len(root.dataList) or it is less than len(root.dataList)
        rounds[roundNum][root.symbol] = root.dataList[rootIdx:newRootIdx].copy()
        operationsIdx += (newRootIdx - rootIdx) * root.elementCount # keeping count of the number of operations we've done
        rootIdx = newRootIdx

        # We remove symbols from the round if they are not the "round roots"
        if root.left.symbol in rounds[roundNum] and not root.left.symbol in outputs:
            del rounds[roundNum][root.left.symbol]
        if root.right.symbol in rounds[roundNum] and not root.right.symbol in outputs:
            del rounds[roundNum][root.right.symbol] 

        if operationsIdx >= layerSize:
            operationsIdx = 0
            roundNum += 1
            rounds[roundNum] = {}
    return [roundNum, operationsIdx]

In [3]:
def readOperationsFile(operations_file):
    ret = []
    inputDic = {}
    with open(operations_file, 'r') as f:
        for line in f:
            if len(line.split()) == 2:
                arr = []
                with open(line.split()[1], 'r') as f2:
                    arr.append(list(i.strip().split(', ') for i in f2.readlines()))
                inputDic[line.split()[0]] = np.array(*arr, dtype=int)
            else:
                ret.append(line.strip())
    return ret, inputDic

dag = {}
outputs = set()

# we read the data as it is passed in the operations file. Note that operations file follows the 
# protocol 1 format
test_operations, input_data = readOperationsFile("./input_files/operations.txt")
print("test operations: ", test_operations)
print("input: ", input_data)
processInputs(input_data, dag)
for i in test_operations:
    solveDAG(i, dag, outputs)
rounds_calculated = {}
calculateRoundParams(dag['operation_4'], 1, 0, rounds_calculated, 4, outputs)
def countNumberOfOperationsRequred(root):
    if root.left == None and root.right == None: return 0
    return root.elementCount * len(root.dataList) + countNumberOfOperationsRequred(root.left) + countNumberOfOperationsRequred(root.right)

test operations:  ['operation_1 add input_var_1 input_var_2', 'operation_2 subtract input_var_1 operation_1', 'operation_3 mul operation_2 input_var_1', 'operation_4 matmul operation_3 identity']
input:  {'input_var_1': array([[1, 2],
       [3, 4]]), 'input_var_2': array([[5, 6],
       [7, 8]]), 'identity': array([[1, 0],
       [0, 1]])}


In [4]:
arbitrum_sepolia_rpc_url = "https://sepolia-rollup.arbitrum.io/rpc"
web3 = Web3(Web3.HTTPProvider(arbitrum_sepolia_rpc_url))
assert web3.is_connected(), "Failed to connect to Arbitrum Sepolia"

In [5]:
solcx.install_solc('0.8.25')
solcx.set_solc_version('0.8.25')

def get_abi_market(file_path, contract_name):
    # Compile the contract
    with open(file_path, 'r') as file:
        contract_source_code = file.read()

    import_remappings = {
        "@openzeppelin/": "../node_modules/@openzeppelin/",
    }

    compiled_sol = solcx.compile_source(contract_source_code, output_values=['abi', 'bin'],
                                        import_remappings=import_remappings,
                                        optimize=True,
                                        optimize_runs=20,
                                        via_ir=True)

    # Extract ABI and bytecode
    contract_interface = compiled_sol['<stdin>:' + contract_name]
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    return bytecode, abi

def build_and_send_tx(func, account, private_key, gas=2000000, gas_price='5', is_constructor=False, tries=3):
    failed = False
    for i in range(tries):
        try:
            if is_constructor:
                tx = func
            else:
                tx = func.build_transaction({
                    'from': account.address,
                    'nonce': web3.eth.get_transaction_count(account.address),
                })
            estimated_gas = int(web3.eth.estimate_gas(tx) * 2)
            base_fee = int(web3.eth.gas_price)
            max_fee_per_gas = base_fee

            tx.update({
                'gas': estimated_gas,
                'maxFeePerGas': max_fee_per_gas,
            })
            
            signed_tx = web3.eth.account.sign_transaction(tx, private_key)
            tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
            time.sleep(4)  # Wait for transaction to be mined
            if failed:
                print("We Failed, but now succeeded on attempt: ", i + 1)
            return web3.eth.get_transaction_receipt(tx_hash)
        except Exception as e:
            if i == tries - 1:
                raise e
            print("Error: ", e)
            print("Will try again in 5 seconds")
            time.sleep(5)
            failed = True

In [6]:
(bytecode_market, market_contract_abi) = get_abi_market("../contracts/ComputationMarket.sol", "ComputationMarket")
market_contract = web3.eth.contract(bytecode=bytecode_market, abi=market_contract_abi)

(bytecode_token, comp_token_abi) = get_abi_market("../contracts/COMPToken.sol", "COMPToken")
comp_contract = web3.eth.contract(bytecode=bytecode_token, abi=comp_token_abi)

(bytecode_handlers, handlers_abi) = get_abi_market("../contracts/HandlerFunctionsCompMarket.sol", "HandlerFunctionsCompMarket")
handler_contract = web3.eth.contract(bytecode=bytecode_handlers, abi=handlers_abi)

(bytecode_NFT, NFT_abi) = get_abi_market("../contracts/COMPNFT.sol", "CompNFT")
NFT_contract = web3.eth.contract(bytecode=bytecode_NFT, abi=NFT_abi)

deployerPrivateKey = "2ae27eeaa8095f56cd7c02adddd144bdc02d67c3d2a890b7f2ee0097cd520934"
deployer = web3.eth.account.from_key(deployerPrivateKey)
deployer_address = deployer.address

# Check deployer's balance
print("Deployers Address: ", deployer_address)
deployer_balance = web3.eth.get_balance(deployer_address)
print(f"Deployer balance: {web3.from_wei(deployer_balance, 'ether')} ETH")

tx = comp_contract.constructor(1000000 * 10 ** 18).build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})
comp_token_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("Comp token address: ", comp_token_contract_address)

tx = handler_contract.constructor().build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})
handler_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("Handler token address: ", handler_contract_address)

tx = NFT_contract.constructor().build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})
nft_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("NFT address: ", nft_contract_address)

tx = market_contract.constructor(comp_token_contract_address, nft_contract_address, handler_contract_address).build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})

market_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("Market address: ", market_contract_address)

with open('./../ComputationMarketABI.json', 'w') as file:
    file.write(json.dumps(market_contract_abi))

with open('./../COMPToken.json', 'w') as file:
    file.write(json.dumps(comp_token_abi))

with open('./../nft.json', 'w') as file:
    file.write(json.dumps(NFT_abi))

with open('./../HandlerContract.json', 'w') as file:
    file.write(json.dumps(handlers_abi))

with open('./../private_keys.json', 'r') as file:
    keys = json.load(file)

time.sleep(2)

metamask_private_keys = [
    keys["consumer"],  # Consumer
    keys["provider"],  # Provider
    keys["verifier1"],  # Verifier1
    keys["verifier2"],  # Verifier2
    keys["verifier3"],  # Verifier3
    keys["verifier4"],  # Verifier4
    keys["verifier5"]   # Verifier5
]

account_roles = [
    "Consumer",
    "Provider",
    "Verifier1",
    "Verifier2",
    "Verifier3",
    "Verifier4",
    "Verifier5"
]

market_contract = web3.eth.contract(address=market_contract_address, abi=market_contract_abi)
comp_token_contract = web3.eth.contract(address=comp_token_contract_address, abi=comp_token_abi)
nft_contract  = web3.eth.contract(address=nft_contract_address, abi=NFT_abi)

func = nft_contract.functions.transferNFTContractOwnership(market_contract_address)
receipt = build_and_send_tx(func, deployer, deployerPrivateKey)
print(f"NFT contract ownership transferred to {market_contract_address}")

def get_balance(account):
    balance = comp_token_contract.functions.balanceOf(account.address).call()
    return web3.from_wei(balance, 'ether')

def print_balance(accounts, roles):
    for account, role in zip(accounts, roles):
        print(f"Balance of {account.address} ({role}): {get_balance(account)} COMPToken")

def approve_tokens(spender_address, amount, account, private_key):
    func = comp_token_contract.functions.approve(spender_address, amount)
    receipt = build_and_send_tx(func, account, private_key)
    #print(f"{account.address} approved {amount} tokens for {spender_address}")

def transfer_tokens(to_account, amount, from_account, private_key):
    func = comp_token_contract.functions.transfer(to_account.address, amount)
    receipt = build_and_send_tx(func, from_account, private_key)
    #print(f"Transferred {amount} tokens from {from_account.address} to {to_account.address}")

def print_request(request_id):
    result = market_contract.functions.requests(request_id).call()
    descriptions = [
        "Consumer address",
        "Payment for provider",
        "Total payment for verifiers",
        "Number of operations",
        "Number of verifiers",
        "Operation file URL",
        "Computation deadline",
        "Verification deadline",
        "Total payment",
        "Completed",
        "Has been computed",
        "Number of verifiers sample size",
        "Main provider address",
        "Time allocated for verification",
        "Layer count",
        "Layer compute index",
        "Round index",
        "State",
        "Stake",
        "Payment per round for verifiers",
        "Total paid for verification",
        "Protocol version",
        "Verifier selection count",
        "First initialised time",
        "Layer size",
        "Hash of input files"
    ]

    for desc, value in zip(descriptions, result):
        print(f"{desc}: {value}")

Deployers Address:  0xA10A627707da9278f07C80983696Dc153E4714aE
Deployer balance: 0.139404394 ETH
Comp token address:  0xfEB93E7A763A565daEEAd8a9313fBb7EaDe7A756
Handler token address:  0xA4330c0dE1F0b33Aa43E99b7d9E46cec376dDB7F
NFT address:  0x8F9a12F82C3d551F928A0f51c7C8fca45f6d4277
Market address:  0x05A03D539cE6235b075e425Bc856097C3E78316d
NFT contract ownership transferred to 0x05A03D539cE6235b075e425Bc856097C3E78316d


In [7]:
def generateHashOfInputFiles(file_paths):
    file_paths.sort()
    sha256_hash = hashlib.sha256()

    for file_path in file_paths:
        with open(file_path, "rb") as f:
            # Read and update hash object with each file's contents
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
    return sha256_hash.digest()

def create_request(account, private_key):
    input_files = ["./input_files/input1.txt", "./input_files/input2.txt", "./input_files/operations.txt", "./input_files/identity.txt"]
    func = market_contract.functions.createRequest(
        100 * 10**18,
        10 * 10**18,
        countNumberOfOperationsRequred(dag["operation_4"]),
        5,
        ["input_files/input1.txt", "input_files/input2.txt", "inputs_files/identity.txt"],
        "input_files/operations.txt",
        int((datetime.now() + timedelta(hours=1000000)).timestamp()),
        int((datetime.now() + timedelta(hours=2000000)).timestamp()),
        100000,
        3,
        1,
        4,
        generateHashOfInputFiles(input_files),
        500 * 10**18
    )
    receipt = build_and_send_tx(func, account, private_key)
    time.sleep(5)
    request_id = market_contract.functions.requestCount().call() - 1
    print(f"Request created with ID: {request_id} by Consumer")
    print_balance([account], ["Consumer"])
    return request_id

def select_request(provider_account, private_key, request_id):
    func = market_contract.functions.selectRequest(request_id)
    receipt = build_and_send_tx(func, provider_account, private_key)
    print(f"Request {request_id} selected by Provider")
    print_balance([provider_account], ["Provider"])

def complete_request(provider_account, private_key, request_id, outputFileURLs):
    func = market_contract.functions.completeRequest(request_id, outputFileURLs)
    receipt = build_and_send_tx(func, provider_account, private_key)
    print(f"Request {request_id} completed by Provider")

def apply_for_verification(verifier_account, private_key, request_id):
    func = market_contract.functions.applyForVerificationForRequest(request_id)
    receipt = build_and_send_tx(func, verifier_account, private_key, 5)
    print(f"Verifier {verifier_account.address} applied for verification")
    print_balance([verifier_account], ["Verifier"])

def trigger_verifier(verifier_account, private_key, request_id):
    func = market_contract.functions.chooseVerifiersForRequestTrigger(request_id)
    receipt = build_and_send_tx(func, verifier_account, private_key, 5)
    print(f"Verifier {verifier_account.address} triggered verifier selection")

def submit_commitment(verifier_account, private_key, request_id, computed_hash):
    func = market_contract.functions.submitCommitment(request_id, computed_hash)
    receipt = build_and_send_tx(func, verifier_account, private_key, 5)
    print(f"Verifier {verifier_account.address} submitted commitment")

def reveal_provider_key_and_hash(provider_account, private_key, request_id, answerHash, privateKeyRand, initialisationVecRand):
    func = market_contract.functions.revealProviderKeyAndHash(
        request_id,
        privateKeyRand,
        initialisationVecRand,
        answerHash 
    )
    receipt = build_and_send_tx(func, provider_account, private_key)
    print(f"Provider revealed key and hash for request {request_id}")

def reveal_commitment(verifier_account, private_key, request_id, agree, answerHash, nonce):
    func = market_contract.functions.revealCommitment(
        request_id,
        agree,
        answerHash,
        nonce 
    )
    receipt = build_and_send_tx(func, verifier_account, private_key)
    print(f"Verifier {verifier_account.address} revealed commitment")

def calculate_majority_and_reward(verifierAccount, private_key, request_id, round_num):
    func = market_contract.functions.calculateMajorityAndReward(request_id, round_num)
    receipt = build_and_send_tx(func, verifierAccount, private_key)
    print(f"Verifier {verifierAccount.address} calculated majority and reward for {request_id}, round {round_num}")
    print_balance([verifierAccount], ["Verifier"])

In [8]:
consumer_account = web3.eth.account.from_key(metamask_private_keys[0])
provider_account = web3.eth.account.from_key(metamask_private_keys[1])
verifier_accounts = [web3.eth.account.from_key(key) for key in metamask_private_keys[2:]]

all_accounts = [consumer_account, provider_account] + verifier_accounts

#reveal_commitment(verifier_accounts[0], verifier_accounts[00].key, 0, True, web3.keccak(text="hashed_answer"), web3.keccak(text="nonce_0"))

print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

# Approve tokens for consumer
approve_tokens(market_contract_address, 500 * 10**18, consumer_account, consumer_account.key)

# Transfer tokens to provider and verifiers
transfer_tokens(provider_account, 1000 * 10**18, consumer_account, consumer_account.key)
for verifier_account in verifier_accounts:
    transfer_tokens(verifier_account, 500 * 10**18, consumer_account, consumer_account.key)

print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

time.sleep(4)
# Create request

Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 1000000 COMPToken
Balance of 0xC7D6129946779d29C35CCd95c984cD1fF0633678 (Provider): 0 COMPToken
Balance of 0x5F373754819cCA00230eCFBE55419d76329b585A (Verifier1): 0 COMPToken
Balance of 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 (Verifier2): 0 COMPToken
Balance of 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 (Verifier3): 0 COMPToken
Balance of 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 (Verifier4): 0 COMPToken
Balance of 0x54CD26745fD46fE30680EE504c952D0B57E377ee (Verifier5): 0 COMPToken
Error:  {'code': -32000, 'message': 'max fee per gas less than block base fee: address 0xA10A627707da9278f07C80983696Dc153E4714aE, maxFeePerGas: 103380000 baseFee: 103570000'}
Will try again in 5 seconds
We Failed, but now succeeded on attempt:  2
Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 996500 COMPToken
Balance of 0xC7D6129946779d29C35CCd95c984cD1fF0633678 (Provider): 1000 COMPToken
Balance of 0x5F373754819cCA0023

In [9]:
request_id = create_request(consumer_account, consumer_account.key)
print_request(request_id)

approve_tokens(market_contract_address, 500 * 10**18, provider_account, provider_account.key)

select_request(provider_account, provider_account.key, request_id)

outputFileURLs = ["./outputs/round1/output_1.txt", "./outputs/round1/output_2.txt", "./outputs/round1/output_3.txt", "./outputs/round1/output_4.txt", "./outputs/round1/output_5.txt"]
complete_request(provider_account, provider_account.key, request_id, outputFileURLs)

Request created with ID: 0 by Consumer
Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 996220 COMPToken
Consumer address: 0xA10A627707da9278f07C80983696Dc153E4714aE
Payment for provider: 100000000000000000000
Total payment for verifiers: 180000000000000000000
Number of operations: 24
Number of verifiers: 5
Operation file URL: input_files/operations.txt
Computation deadline: 5325663946
Verification deadline: 8925667546
Total payment: 280000000000000000000
Completed: False
Has been computed: False
Number of verifiers sample size: 3
Main provider address: 0x0000000000000000000000000000000000000000
Time allocated for verification: 100000
Layer count: 6
Layer compute index: 0
Round index: 0
State: 0
Stake: 500000000000000000000
Payment per round for verifiers: 10000000000000000000
Total paid for verification: 0
Protocol version: 1
Verifier selection count: 0
First initialised time: 0
Layer size: 4
Hash of input files: b'\xec\x9f9\xf4H\xb0\x93>\xfe$j\x9d2\xac\x0f\xf0\xfeO\x

In [10]:
print_balance([market_contract], ["market"])
print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

Balance of 0x05A03D539cE6235b075e425Bc856097C3E78316d (market): 780 COMPToken
Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 996220 COMPToken
Balance of 0xC7D6129946779d29C35CCd95c984cD1fF0633678 (Provider): 500 COMPToken
Balance of 0x5F373754819cCA00230eCFBE55419d76329b585A (Verifier1): 500 COMPToken
Balance of 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 (Verifier2): 500 COMPToken
Balance of 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 (Verifier3): 500 COMPToken
Balance of 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 (Verifier4): 500 COMPToken
Balance of 0x54CD26745fD46fE30680EE504c952D0B57E377ee (Verifier5): 500 COMPToken


In [11]:
def isVerifierChosenForRound(verifier_account, request_id, round_number):
    return market_contract.functions.isVerifierChosenForRound(request_id, round_number + 1, verifier_account.address).call(
        {
            'from': verifier_account.address
        })

In [12]:
# Function to convert integers to 32-byte representation
def int_to_32bytes(i):
    # Handle negative integers using two's complement
    if i < 0:
        i = (1 << 256) + i  # Convert to two's complement 256-bit integer
    return i.to_bytes(32, byteorder='big')    
# Function to convert 32-byte representation back to integer
def bytes_to_int(b):
    i = int.from_bytes(b, byteorder='big')
    # Check if the integer is negative by checking the most significant bit (MSB)
    if i >= (1 << 255):  # If the MSB is set
        i -= (1 << 256)  # Convert back to a negative number using two's complement
    return i

# Function to encrypt data using AES-256 in CTR mode
def encrypt_round_data(data, keys, ctr, round_num):
    # Derive a 256-bit AES key from the given key
    private_key = sha256(keys[round_num].encode('utf-8')).digest()
    # Initialize AES cipher in CTR mode
    cipher = AES.new(private_key, AES.MODE_CTR, counter=ctr)
    # Encrypt the data
    encrypted_data = cipher.encrypt(data)
    return encrypted_data

# Example of how to process rounds and encrypt them
def encrypt_rounds(rounds, keys):
    encrypted_rounds = {}
    for round_num, round_data in rounds.items():
        ctr = Counter.new(128)
        encrypted_rounds[round_num] = {}
        for symbol in sorted(round_data.keys()):
            encrypted_rounds[round_num][symbol] = encrypt_round_data(b''.join(int_to_32bytes(i) for i in round_data[symbol]), keys[round_num], ctr, round_num)
    return encrypted_rounds

# Function to decrypt data using AES-256 in CTR mode
def decrypt_round_data(encrypted_data, keys, ctr, round_idx):
    # Derive a 256-bit AES key from the given key
    private_key = sha256(keys[round_idx].encode('utf-8')).digest()
    # Initialize AES cipher in CTR mode with the same counter used during encryption
    cipher = AES.new(private_key, AES.MODE_CTR, counter=ctr)
    # Decrypt the data
    decrypted_data = cipher.decrypt(encrypted_data)
    return decrypted_data

# Example of how to process decrypted rounds
def decrypt_rounds(encrypted_rounds, keys):
    decrypted_rounds = {}
    for round_num, encrypted_data in encrypted_rounds.items():
        ctr = Counter.new(128)
        decrypted_rounds[round_num] = {}
        for symbol in sorted(encrypted_rounds[round_num].keys()):
            decrypted_data = decrypt_round_data(encrypted_data[symbol], keys, ctr, round_num)
            decrypted_rounds[round_num][symbol] = [bytes_to_int(decrypted_data[i:i+32]) for i in range(0, len(decrypted_data), 32)]
    return decrypted_rounds

In [13]:
# since this is a simple example we do not calculate the next round values using the previous round values, howver
# this is what will need to be done for longer processes

def generateKey():
    # Generate a random 256-bit number as our key
    random_256bit_number = secrets.randbits(256)
    # Convert to a 32-byte (256-bit) hexadecimal string if needed
    random_256bit_hex = hex(random_256bit_number)[2:]  # Remove the '0x' prefix
    # Ensure it's 64 hex characters (32 bytes) long by padding with zeros if necessary
    random_256bit_hex = random_256bit_hex.zfill(64)
    key = random_256bit_hex
    return key, random_256bit_number

def generateAnswerHashForRound(roundIdx, rounds_val):
    v = list(rounds_val[roundIdx].items())
    v.sort(key=lambda x: x[0])
    s = [scalar for x in v for scalar in x[1]]
    return Web3.solidity_keccak(['int32']*len(s), s)

def generateRoundFile(rounds_vals):
    kk = [generateKey() for i in rounds_vals]
    keys = [x[0] for x in kk]
    keysUnhexed = [x[1] for x in kk]
    e = encrypt_rounds(rounds_vals, keys)
    for k, v in e.items():
        #Generate the round files
        #return the different keys and IV we used to encrypt the round files
        # create a file called round_number.txt
        f = open(f"./outputs/round_{k}.txt", "w")
        for a, b in v.items():
            f.write(a + ":" + str(b))
        f.close()
    return keysUnhexed 

keys_iv = generateRoundFile(rounds_calculated)

In [14]:
answer_hash = generateAnswerHashForRound(1, rounds_calculated)
print(answer_hash)

b'C\x0e\x01\x98k\xd0\xb0\xda\xa9#Bs"rtRf]\x02^\xf44\xffS\xdc\x17\x97\xe1\x95)d\x18'


In [15]:
input_files = ["./input_files/input1.txt", "./input_files/input2.txt", "./input_files/operations.txt", "./input_files/identity.txt"]

for round_number in range(6):

    for verifier_account in verifier_accounts:
        approve_tokens(market_contract_address, 10 * 10**18, verifier_account, verifier_account.key)
    time.sleep(5)

    # Apply for verification as verifiers
    for verifier_account in verifier_accounts:
        apply_for_verification(verifier_account, verifier_account.key, request_id)
    time.sleep(10)

    for verifier_account in verifier_accounts:
        trigger_verifier(verifier_account, verifier_account.key, request_id)
    time.sleep(10)

    print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

    print(f"Starting round {round_number + 1}")

    for verifier_account in verifier_accounts:
        
        answer_hash = generateAnswerHashForRound(round_number + 1, rounds_calculated) # since our test case is small, we will just calculate the entire DAG and use that as our computation
        nonce = Web3.keccak(text=f"nonce_{verifier_account.address}")

        verifierChosen = isVerifierChosenForRound(verifier_account, request_id, round_number)

        if verifierChosen:
            # Concatenate the byte representations of the inputs
            concatenated = Web3.solidity_keccak(
                ['bytes32', 'bytes32', 'address'],
                [answer_hash, nonce, verifier_account.address]
            )
            submit_commitment(verifier_account, verifier_account.key, request_id, concatenated)

    provider_answer_hash = generateAnswerHashForRound(round_number + 1, rounds_calculated)
    provider_key = keys_iv[round_number] # we use the private key for this round
    initialisationVector = 128 # for this example we just use a constant number for our initialisation vector
    reveal_provider_key_and_hash(provider_account, provider_account.key, request_id, provider_answer_hash, provider_key, initialisationVector)

    for verifier_account in verifier_accounts:
        answer_hash = generateAnswerHashForRound(round_number + 1, rounds_calculated)
        nonce = web3.keccak(text=f"nonce_{verifier_account.address}")

        verifierChosen = isVerifierChosenForRound(verifier_account, request_id, round_number)

        if verifierChosen:
            reveal_commitment(verifier_account, verifier_account.key, request_id, True, answer_hash, nonce)

    for verifier_account in verifier_accounts:
        if isVerifierChosenForRound(verifier_account, request_id, round_number):
            calculate_majority_and_reward(verifier_account, verifier_account.key, request_id, round_number + 1)

    print_request(request_id)

    print(f"Balances after round {round_number + 1}:")
    print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

Verifier 0x5F373754819cCA00230eCFBE55419d76329b585A applied for verification
Balance of 0x5F373754819cCA00230eCFBE55419d76329b585A (Verifier): 490 COMPToken
Verifier 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 applied for verification
Balance of 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 (Verifier): 490 COMPToken
Verifier 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 applied for verification
Balance of 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 (Verifier): 490 COMPToken
Verifier 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 applied for verification
Balance of 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 (Verifier): 490 COMPToken
Verifier 0x54CD26745fD46fE30680EE504c952D0B57E377ee applied for verification
Balance of 0x54CD26745fD46fE30680EE504c952D0B57E377ee (Verifier): 490 COMPToken
Verifier 0x5F373754819cCA00230eCFBE55419d76329b585A triggered verifier selection
Verifier 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 triggered verifier selection
Error:  {'code': -32000, 'message': 'gas required exc

In [16]:
print("Done!")
print("Final Request State:")
print_request(request_id)

Done!
Final Request State:
Consumer address: 0xA10A627707da9278f07C80983696Dc153E4714aE
Payment for provider: 100000000000000000000
Total payment for verifiers: 180000000000000000000
Number of operations: 24
Number of verifiers: 5
Operation file URL: input_files/operations.txt
Computation deadline: 5325663946
Verification deadline: 8925667546
Total payment: 280000000000000000000
Completed: True
Has been computed: True
Number of verifiers sample size: 3
Main provider address: 0xC7D6129946779d29C35CCd95c984cD1fF0633678
Time allocated for verification: 100000
Layer count: 6
Layer compute index: 6
Round index: 6
State: 6
Stake: 500000000000000000000
Payment per round for verifiers: 10000000000000000000
Total paid for verification: 180000000000000000000
Protocol version: 1
Verifier selection count: 5
First initialised time: 1725663973
Layer size: 4
Hash of input files: b'\xec\x9f9\xf4H\xb0\x93>\xfe$j\x9d2\xac\x0f\xf0\xfeO\x86wk\x18\xba\xc9\n1}\xe4\xbai\xfc\xc3'
